In [ ]:
import numpy as np
from simple import nmf_son

np.random.seed(42)
np.set_printoptions(precision=3)

In [ ]:
def load_results(filepath):
    data = np.load(filepath)
    return data['Wb'], data['Hb'], data['Wl'], data['Hl'], data['fscores'], data['gscores'], data['lambda_vals']

In [ ]:
X = np.load('urban/2022_11_17/small/X.npz')['X']

In [ ]:
data = np.load('urban/2022_11_17/small/r6_ini.npz')
ini_W = data['W']
ini_H = data['H']

In [ ]:
rank = 6
iters = 2500

save_filepath = 'urban/2022_11_21/small/r{}-l{}-it{}.npz'

In [ ]:
# reg_vals = 10 ** np.linspace(-3, 3, 20)
reg_vals = [0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.25, 0.5, 0.75, 1, 2, 5, 10, 25, 50, 100, 200, 500, 1000]
it_ckpts = [200, 500, 1000, 2000, 3000, 4000, 5000]

In [ ]:
reg_file_dict = {x: None for x in reg_vals}

In [ ]:
for k in range(len(it_ckpts)):
    for reg_val in reg_vals:
        if reg_file_dict[reg_val]:
            Wb, Hb, Wl, Hl, fscores, gscores, lambda_vals = load_results(reg_file_dict[reg_val])

            Wb, Hb, Wl, Hl, new_fscores, new_gscores, new_lambda_vals = nmf_son(X, Wl, Hl, _lambda=reg_val, itermax=it_ckpts[k] - it_ckpts[k-1], scale_lambda=True)

            fscores = np.append(fscores[:-1], new_fscores)
            gscores = np.append(gscores[:-1], new_gscores)
            lambda_vals = np.append(lambda_vals[:-1], new_lambda_vals)
            print(2)
        else:
            Wb, Hb, Wl, Hl, fscores, gscores, lambda_vals = nmf_son(X, ini_W.copy(), ini_H.copy(), _lambda=reg_val, itermax=it_ckpts[k], scale_lambda=True)
            print(1)
        tmp_filepath = save_filepath.format(rank, str(reg_val).replace('.', '_'), it_ckpts[k])
        with open(tmp_filepath, 'wb') as fout:
            np.savez_compressed(fout, Wb=Wb, Hb=Hb, Wl=Wl, Hl=Hl, fscores=fscores, gscores=gscores, lambda_vals=lambda_vals)
        reg_file_dict[reg_val] = tmp_filepath
        print(reg_val, it_ckpts[k], 'complete')
